<a href="https://colab.research.google.com/github/CaretJuice/extract-ga4-event-params-bigquery/blob/main/extract_ga4_event_params_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Variables
spreadsheet = ''  # using the connect by URL method so put the full URL of the sheet that you are using
source_project = ''
source_dataset = ''
source_table = ''

# string of parameter names formatted to go in a WHERE event_parameter.key NOT IN ('language','page_location')
# intent of this sheet is to surface custom parameters so excluding more than Google's officially documented list of default parameters
# https://support.google.com/analytics/answer/9234069?hl=en&ref_topic=13367566&sjid=4176436952221836768-NC
# leave empty string to select all parameters
parameters_to_exclude = """
  'language', 'page_location', 'page_referrer', 'page_title', 'screen_resolution', 'engagement_time_msec', 
  'source', 'medium', 'campaign', 'batch_ordering_id', 'ga_session_number', 'ga_session_id', 'page_title', 
  'screen_name', 'session_engaged', 'debug_mode', 'ignore_referrer', 'batch_page_id', 'engaged_session_event'
"""

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table


import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Authenticated
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
event_params = client.query('''
    select
      event_name
      , ep.key
      , count(ep.value.string_value) as string_values
      , count(ep.value.int_value) as int_values
      , count(ep.value.float_value) as float_values
      , count(ep.value.double_value) as double_values
    from {sp}.{sd}.{st}
    cross join
      unnest(event_params) ep
    where ep.key not in ({pe})
    group by event_name, ep.key
    union all
    select
      'user_property' as event_name
      , up.key
      , count(up.value.string_value) as string_values
      , count(up.value.int_value) as int_values
      , count(up.value.float_value) as float_values
      , count(up.value.double_value) as double_values
    from {sp}.{sd}.{st}
    cross join
      unnest(user_properties) up
    group by event_name, up.key
  '''.format(sp=source_project, sd=source_dataset, st=source_table, pe=parameters_to_exclude)).to_dataframe()

event_params

In [ ]:
from gspread_dataframe import set_with_dataframe

ss = gc.open_by_url(spreadsheet)
try:
    ws = ss.worksheet(source_dataset)
except:
    ws = ss.add_worksheet(title=source_dataset, rows='1', cols ='1')
set_with_dataframe(ws, event_params )
#restore the header text
ws.update('A1', source_dataset)

first_visit

                  key  string_values  int_values  float_values  double_values
0         form_length              0           1             0              0
1       batch_page_id              0           1             0              0
2     ignore_referrer              1           0             0              0
3          page_title              1           0             0              0
4       page_referrer              1           0             0              0
5    form_destination              1           0             0              0
6       page_location              1           0             0              0
7   ga_session_number              0           1             0              0
8     session_engaged              1           0             0              0
9   batch_ordering_id              0           1             0              0
10      ga_session_id              0           1             0              0


session_start

                  key  string_valu